In [1]:
import numpy as np
import cv2
from Undistort import Undistort
from Threshold import Threshold
from Warp import Warp
from Polyfit import Polyfit
from Polydraw import Polydraw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.video.io.VideoFileClip import VideoFileClip
from IPython.display import HTML

print('Loaded Modules!!')

Loaded Modules!!


In [2]:
undistort = Undistort()
polydraw = Polydraw()
polyfit = Polyfit()

In [3]:
src = np.float32([
    [580, 460],
    [700, 460],
    [1040, 680],
    [260, 680],
])

dst = np.float32([
    [260, 0],
    [1040, 0],
    [1040, 720],
    [260, 720],
])

In [4]:
warp = Warp(src, dst)
undistort.calibratie_camera()

Working on Image :  ./camera_cal/calibration10.jpg
Working on Image :  ./camera_cal/calibration11.jpg
Working on Image :  ./camera_cal/calibration12.jpg
Working on Image :  ./camera_cal/calibration13.jpg
Working on Image :  ./camera_cal/calibration14.jpg
Working on Image :  ./camera_cal/calibration15.jpg
Working on Image :  ./camera_cal/calibration16.jpg
Working on Image :  ./camera_cal/calibration17.jpg
Working on Image :  ./camera_cal/calibration18.jpg
Working on Image :  ./camera_cal/calibration19.jpg
Working on Image :  ./camera_cal/calibration2.jpg
Working on Image :  ./camera_cal/calibration20.jpg
Working on Image :  ./camera_cal/calibration3.jpg
Working on Image :  ./camera_cal/calibration6.jpg
Working on Image :  ./camera_cal/calibration7.jpg
Working on Image :  ./camera_cal/calibration8.jpg
Working on Image :  ./camera_cal/calibration9.jpg


In [5]:
def process_images_test(base_image, name):

    img_undist = undistort.undistort(base_image)
    mpimg.imsave('./output_images/undistorted_{}'.format(name), img_undist)

    threshold = Threshold()
    img = threshold.combined_threshold(img_undist)
    mpimg.imsave('./output_images/thresholded_{}'.format(name), img, cmap="gray")

    img = warp.warp(img)
    mpimg.imsave('./output_images/warped_{}'.format(name), img, cmap="gray")

    left_fit, right_fit = polyfit.polyfit(img, visualize=True)

    img = polydraw.draw(img_undist, left_fit, right_fit, warp.Minv)

    mpimg.imsave('./output_images/final_{}'.format(name), img, cmap="gray")
    # Measure curvature
    lane_curve, car_position = polyfit.curvature(img)
    font = cv2.FONT_HERSHEY_SIMPLEX

    if car_position > 0:
        cv2.putText(img, 'Radius of curvature (Left)  = %.2f m' % (lane_curve), (10, 40), font, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        pos_text = '{}m right of center'.format(car_position)
    else:
        cv2.putText(img, 'Radius of curvature (Right) = %.2f m' % (lane_curve), (10, 70), font, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        pos_text = '{}m left of center'.format(abs(car_position))

    cv2.putText(img, "Lane curve: {}m".format(lane_curve.round()), (10, 40), font, 1,
                color=(255, 255, 255), thickness=2)
    cv2.putText(img, "Car is {}".format(pos_text), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color=(255, 255, 255),
                thickness=2)
    mpimg.imsave('./output_images/annotated_{}'.format(name), img)

In [6]:
test_images = ['straight_lines1.jpg', 'straight_lines2.jpg', 'test1.jpg', 'test2.jpg', 'test3.jpg', 'test4.jpg', 'test5.jpg', 'test6.jpg']
for image in test_images:
    img = mpimg.imread('./test_images/' + image)
    process_images_test(img, image)

604.582419912 m 489.34174028 m
452.81367643 m 601.121487301 m
584.9237814 m 581.622525651 m
503.330573588 m 517.408988609 m
578.034638667 m 482.271073253 m
541.813039586 m 547.02780876 m
598.772457126 m 595.638373885 m
570.110966595 m 482.496863995 m


In [7]:
def process_image(base_image):
    img_undist = undistort.undistort(base_image)
    threshold = Threshold()
    img = threshold.combined_threshold(img_undist)
    img = warp.warp(img)
    left_fit, right_fit = polyfit.polyfit(img, visualize=False)
    img = polydraw.draw(img_undist, left_fit, right_fit, warp.Minv)
    # Measure curvature
    lane_curve, car_position = polyfit.curvature(img)
    font = cv2.FONT_HERSHEY_SIMPLEX

    if car_position > 0:
        cv2.putText(img, 'Radius of curvature (Left)  = %.2f m' % (lane_curve), (10, 40), font, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        pos_text = '{}m right of center'.format(car_position)
    else:
        cv2.putText(img, 'Radius of curvature (Right) = %.2f m' % (lane_curve), (10, 70), font, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        pos_text = '{}m left of center'.format(abs(car_position))

    cv2.putText(img, "Lane curve: {}m".format(lane_curve.round()), (10, 40), font, 1,
                color=(255, 255, 255), thickness=2)
    
    cv2.putText(img, "Car is {}".format(pos_text), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color=(255, 255, 255),
                thickness=2)
    return img

In [8]:
test_output = 'project_video_test.mp4'
clip_test = VideoFileClip('project_video.mp4')
test_clip = clip_test.fl_image(process_image).subclip(38,43)  # NOTE: this function expects color images!!
test_clip.write_videofile(test_output, audio=False)

466.338407292 m 500.750139818 m
582.739664411 m 437.441086171 m
[MoviePy] >>>> Building video project_video_test.mp4
[MoviePy] Writing video project_video_test.mp4


  1%|          | 1/126 [00:00<00:23,  5.31it/s]

516.950432468 m 616.307651638 m


  2%|▏         | 2/126 [00:00<00:23,  5.30it/s]

461.523188552 m 545.762226527 m


...

514.014575331 m 545.390586452 m
614.964147217 m 595.03208599 m


 98%|█████████▊| 124/126 [00:22<00:00,  5.37it/s]

536.523602164 m 486.712715033 m
506.267498403 m 620.614099949 m


 99%|█████████▉| 125/126 [00:22<00:00,  5.34it/s]


560.180923144 m 485.774772392 m
[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_test.mp4 



In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(test_output))

In [10]:
white_output = 'project_video_done_2.mp4'
clip1 = VideoFileClip('project_video.mp4')
white_clip = clip1.fl_image(process_image)  # NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)

626.535751385 m 566.053287501 m
[MoviePy] >>>> Building video project_video_done_2.mp4
[MoviePy] Writing video project_video_done_2.mp4


  0%|          | 1/1261 [00:00<03:45,  5.59it/s]

497.743309221 m 626.286756629 m


  0%|          | 2/1261 [00:00<03:39,  5.73it/s]

561.474193563 m 537.997545751 m


...

100%|█████████▉| 1259/1261 [04:48<00:00,  4.46it/s]

521.936918233 m 574.76927827 m


100%|█████████▉| 1260/1261 [04:48<00:00,  4.35it/s]

706.943189909 m 618.04965938 m


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_done_2.mp4 



In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [12]:
challenge = 'challenge_video_done_2.mp4'
clip1 = VideoFileClip('challenge_video.mp4')
white_clip = clip1.fl_image(process_image)  # NOTE: this function expects color images!!
white_clip.write_videofile(challenge, audio=False)

587.126018425 m 568.517910689 m
[MoviePy] >>>> Building video challenge_video_done_2.mp4
[MoviePy] Writing video challenge_video_done_2.mp4


  0%|          | 1/485 [00:00<01:15,  6.41it/s]

590.11339151 m 554.913210873 m


  0%|          | 2/485 [00:00<01:15,  6.42it/s]

537.640641386 m 568.104174173 m


...

100%|█████████▉| 483/485 [01:29<00:00,  5.56it/s]

529.572334784 m 612.979175769 m


100%|██████████| 485/485 [01:30<00:00,  5.41it/s]

546.59234778 m 523.405051914 m
583.868848976 m 456.617919721 m


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_done_2.mp4 



In [13]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge))

In [14]:
harder_output = 'harder_challenge_video_done_2.mp4'
clip1 = VideoFileClip('harder_challenge_video.mp4')
white_clip = clip1.fl_image(process_image)  # NOTE: this function expects color images!!
white_clip.write_videofile(harder_output, audio=False)

594.115763989 m 653.976557865 m
[MoviePy] >>>> Building video harder_challenge_video_done_2.mp4
[MoviePy] Writing video harder_challenge_video_done_2.mp4


  0%|          | 1/1200 [00:00<03:31,  5.67it/s]

620.901462317 m 503.054811997 m


  0%|          | 2/1200 [00:00<03:31,  5.67it/s]

475.460809321 m 454.263924592 m


...

100%|█████████▉| 1198/1200 [04:30<00:00,  4.45it/s]

531.66806982 m 543.788099318 m


100%|█████████▉| 1199/1200 [04:30<00:00,  4.46it/s]

591.520272646 m 568.34271487 m


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_video_done_2.mp4 



In [15]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_output))